In [ ]:
import pandas as pd

# Load the datasets (You should replace these with the actual paths to your files)
df_part1 = pd.read_csv('archive/item_properties_part1.csv')
df_part2 = pd.read_csv('archive/item_properties_part2.csv')

# Combine both parts of item properties
df_properties = pd.concat([df_part1, df_part2])

In [ ]:
# Explore available properties
unique_properties = df_properties['property'].unique()
print("Unique properties:", unique_properties)

In [ ]:
# Extract rows where the value looks like a number (e.g., price)
df_numeric_properties = df_properties[df_properties['value'].str.startswith('n', na=False)]

# Check how many unique item ids have numerical values
print(df_numeric_properties[['itemid', 'property', 'value']].drop_duplicates().head(10))

# Look for potential price columns by observing the value patterns
# If necessary, filter for potential "price" properties by looking at large numbers.


In [ ]:
# Extract rows where property might represent availability or stock (likely binary)
df_stock_properties = df_properties[df_properties['property'] == 'available']

# Check the values
print(df_stock_properties[['itemid', 'value']].drop_duplicates().head(10))


In [34]:
# Pivot the data using identified properties
df_pivot = df_properties.pivot_table(index='itemid', columns='property', values='value', aggfunc='first').reset_index()

# Example: If we know '888' is price and 'available' is stock
df_pivot.rename(columns={
    '888': 'price',  # Hypothetical mapping
    'available': 'stock',  # Stock column
    'categoryid': 'category'
}, inplace=True)

# Fill missing values
df_pivot['price'] = df_pivot['price'].fillna(0).astype(float)  # Default price to 0.0
df_pivot['stock'] = df_pivot['stock'].fillna(1).astype(int)  # Default stock to 1 (available)

# Assuming 'itemid' acts as product_id
df_products = df_pivot[['itemid', 'price', 'stock', 'category']]
df_products.rename(columns={'itemid': 'product_id'}, inplace=True)

# Save the cleaned product dataset for MySQL import
df_products.to_csv('products_clean.csv', index=False)


ValueError: could not convert string to float: '172646 1154859'

In [ ]:
# From this, identify columns related to 'name', 'price', 'stock', and 'description'
# We'll extract the most recent value for each property based on the timestamp
df_properties.sort_values(by=['itemid', 'timestamp'], ascending=[True, False], inplace=True)

In [ ]:
# Pivot the data so that each item has its properties as columns
df_pivot = df_properties.pivot_table(index='itemid', columns='property', values='value', aggfunc='first').reset_index()

In [ ]:
# Select relevant columns (assuming 'name', 'price', and 'stock' are part of the available properties)
# If the properties are not explicitly named, we may have to use placeholders or infer from other details.
df_products = df_pivot[['itemid', 'name', 'description', 'price', 'stock']]

In [ ]:
# Fill missing values
df_products['description'].fillna('No description available', inplace=True)
df_products['price'].fillna(0.0, inplace=True)  # Default price is 0.0 if missing
df_products['stock'].fillna(100, inplace=True)  # Default stock level is 100 if missing